# 15. Topic Modelling

The bag of words model (BoW), represented as a document-term matrix (DTM) is useful to compare documents with each other, or compare a query with candidate documents. 

However, this model produces high dimensional data and sparse data, and does little to summarise the content to get to the meaning of a document. Topic modelling is an unsupervised learning technique that address this. It allows us to tag documents based on their affinity with the various topics.

**Topic models** generate sophisticated, interpretable text features that can be a first step towards extracting signals from a large collection of documents. Its applications include unsupervised discovery of potentially insightful themes that can be used.

## Learning latent topics: goals & approaches
Topic modelling aims to discover hidden topics or themes across documents that capture semantic information beyond individual words. It goes beyond lexical analysis which only analyses words, towards semantic analysis which infers the writer's intent. In other words, topic modelling is used to summarise large collections of documents to facilitate organisation, management and search and recommendations.

They aim to address the curse of dimensionality that plague the BoW model. Furthermore, it addresses the polysematic problem that the BoW model cannot capture. Topic modelling is an unsupervised machine learning technique to extract hidden themes from documents.

We move beyond linear algebra to hierarchical probabilistic models for this type of problem. It assumes that there is an explicit document generation process and provide algorithms to reverse engineer this process to recover the underlying topics.

The key models for discussion in this chapter are:

|Model|Description|
|-|:--|
|Latent Semantic Indexing (LSI)|Reduce the word space dimensionality to capture semantic document-term relationships|
|Probabilistic Latent Semantic Analysis (pLSA)|Reverse-engineer a process that assumes words generate a topic and documents are a mix of topics|
|Latent Dirichlet Allocation (LDA)|Adds a generative process for documents: a three-level hierarchical Bayesian model|

## Latent semantic indexing
Latent Semantic Indexing (LSI) or latent semantic analysis improves on the BoW model which omitted synonyms of query terms. Its goal was to model the relationships between documents and terms so it could predict that a term should be associated with a document even though it does not because of the variability in word use.

LSI uses Singular Value Decomposition (SVD) to find the best lower-rank document term matrix approximation using $k$ singular values and vectors. In other words, LSI uses some dimensionality reduction techniques.

SVD identifies a set of uncorrelated indexing variables or factors that represent each term and document by its vector of factor values.

The LSI decomposition of the DTM is as follows:
- Given a $M \times T$

$$\begin{pmatrix}&\cdots&\\\vdots&\text{DTM}&\vdots\\&\cdots&\end{pmatrix} = \begin{pmatrix}&\cdots&\\\vdots&U&\vdots\\&\cdots&\end{pmatrix} \begin{bmatrix}\ddots&&\\&\Sigma&\\&&\ddots\end{bmatrix}\begin{pmatrix}&\cdots&\\\vdots&V^T&\vdots\\&\cdots&\end{pmatrix} $$

### Implementation
(See Notebook 15-1 demo 1)

### Strengths & limitations
LSI removes noise and mitigates the curse of dimensionality. It also captures some semantic aspects like synonymy and clusters both documents and terms via their topic associations.

However, the results of LSI are difficult to interpret because topics are word vectors with both positive and negative entries. Also, there is no underlying model that would permit the evaluation of fit or provide guidance when selecting the number of dimensions or topics to use.

## Probabilistic latent semantic analysis

Probabilistic latent semantic analysis (pLSA) takes a statistical perspective on LSI/LSA and creates a generative model to address the lack of theoretical underpinnings of LSA.

pLSA explicitly models the probability word $w$ appearing in document $d$ as described by the DTM as a mixture of conditionally independent, multinomial distributions that involve topics $t$.

There are both symmetric and asymmetric formulations of how word-document occurrences came about. 

The symmetric formulation assumes that both words and documents are generated by the latent topic classes.

$$P(w,d) = \sum_t P(d|t) \cdot P(w|t)$$

In contrast, the asymmetric formulation assumes that topics are selected given the document and words result in a second step given the topic.

$$P(w,d) = P(d) \sum_t P(t|d) \cdot P(w|t) $$

The number of topics is a hyperparameter chosen before training.

### Implementation
(See notebook 15-1 demo 2)

pLSA is equivalent to non-negative matrix factorization (NMF), so we can use the NMF class to implement pLSA.

### Strengths & limitations
With a probability model, we can now compare the performance of different models by evaluating the probability the assign to new documents given the parameters learned during training. Also, the results have a clear probabilistic interpretation. Also, pLSA captures more semantic information, including polysemy.

However, pLSA increases the computational complexity compared to LSI, and the algorithm may yield a local instead of a global maximum. Also, it does not yield a generative model for new documents because it takes them as giv
en.

## Latent Dirichlet allocation

Latent Dirichlet alocation (LDA) extends pLSA by adding a generative process for topics. It is the most popular topic model because it tends to produce meaningful topics that humans can relate to.

### How LDA works
LDA is a hierarchical Bayesian model that assumes topics are probability distributions over words, and documents are distributions over topics. Specifically, the model assumes that topics follow a sparse Dirichlet distribution, which implies that documents reflect only a small set of topics and topics use only a limited number of terms frequently.

### The Dirichlet distribution
The Dirichlet distribution produces probability vectors that can be used as a discrete probability distribution. It has a parameter of $\alpha$ of positive real value that controls the concentration of probabilities. An $\alpha$ close to $0$ means only a few values will be positive and receive most of the probability mass.

### The generative model
The LDA topic model assumes the following generative process when an author adds an article to a body of documents:

1. Randomly mix a small subset of topics with proportions defined by the Dirichlet distribution
2. For each word in the text to be generated, select one of the topics according to the document-topic probabilities defined in (1).
3. From the Topic selected in (2), select a word from the topic's word list according to the topic-word probabilities.

As a result, the article content depends on the weight of each topic and the terms that make up each topic. The Dirichlet distribution governs the selection of topics for documents and words for topics. It encodes the idea that a document only covers a few topics, while each topic uses only a small number of words frequently.

### Reverse engineering the process
The generative process is fictional but is useful because it permits the recovery of the various (topic & word-topic) distributions. The LDA algorithm reverses engineers the work of the imaginary author and arrives at a summary of the document-topic-word relationships that consisely describes:

- The percentage contribution of each topic to a document
- The probabilistic association of each word with a topic

LDA solves the Bayesian inference problem of recovering the distributions from the corpus and the words they contain by reverse engineering the assumed content generation process.

### How to evaluate LDA topics
Two options to evaluate LDA results are perplexity which evaluates the model on unseen documents, and topic coherence metrics which aim to evaluate the sementic quality of the uncovered patterns.

#### Perplexity
Perplexity, when applied to LDA, measures how well the topic-word probability distribution recovered by the model predicts a sample of unseen text documents. It is based on the entropy $H(p)$ of this distribution $p$ and is computed w.r.t. the set of tokens $w$.

$$2^{h(p)} = 2^{\sum_w p(w) \log_2 p(w)}$$

Values closer to $0$ imply the distribution is better at predicting the sample.

#### Topic coherence
Topic coherence measures the semantic consistency of the topic model results, that is, whether humans would perceive the words and their probabilities associated with topics as meaningful. It scores each topic by measuring the degree of semantic similarity between the words most relevant to the topic. Specifically, coherence measures are based on the probability of observing the set of words $W$ that defines a topic together.

There are two measures of coherence, namely the UMass and UCI metrics.

The UCI metric defines a word pair's score to be the sum of the **pointwise mutual information (PMI)** between two distinct pairs of (top) topic words $w_i,w_j \in w$ and a smoothing factor $\epsilon$

$$\text{Coherence}_\text{UCI} = \sum_{(w_i,w_j) \in W} \log \frac{p(w_i,w_j) + \epsilon}{p(w_i)p(w_j)}$$

The probabilities are computed from word co-occurrence frequencies in a sliding window over an external corpus, so this can be thought of as an external comparison to semantic ground truth.

In contrast, the UMass metric uses the co-occurences in a number of documents $D$ from the training corpus to compute a coherence score:

$$\text{Coherence}_\text{UMass} = \sum_{(w_i,w_j) \in W} \log \frac{D(w_i,w_j) + \epsilon}{D(w_j)}$$

Rather than comparing the model result to extrinsic ground truth, this measure reflects intrisic coherence. In both cases, values closer to $0$ imply that a topic is more coherent.

### Implementation
(See notebook 15-2)